In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from prod_details import Parameters
pm = Parameters()

In [2]:
# Load the data
warehousing_handlingOut_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling Out')
warehousing_handlingIn_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling In')
warehousing_handlingIn_data.index = warehousing_handlingOut_data.DC
warehousing_storageCost = pd.read_excel('Warehousing.xlsx', sheet_name='Storage')
demand_data = pd.read_excel('Demand Forecast.xlsx')
unit_data = pd.read_excel('Product Data per State.xlsx')
unit_data.index = unit_data.state

product_data = pd.read_excel('Product Master Data.xlsx')
product_data.index = product_data.COMM_NAME

warehousing_handlingOut_data.index = warehousing_handlingOut_data.DC
warehousing_handlingIn_data.index = warehousing_handlingOut_data.DC
warehousing_storageCost.index = warehousing_storageCost.DC
demand_data.index = demand_data.state

In [22]:
# Can be deleted if it is not too necessary
as_is_dc = as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

dc_product_demand_box = pm.product_demand_per_dc(as_is_dc, demand_data, product_data)
dc_product_demand_container = pm.product_demand_per_dc(as_is_dc, demand_data, product_data, unit_in_box=False)

In [9]:
volume_dc = pm.volume_per_dc(dc_product_demand_box)
volume_dc

{'NY': {'blender': 87,
  'swing': 3,
  'chair': 109,
  'scooter': 2,
  'skiprope': 585},
 'ND': {'blender': 13, 'swing': 1, 'chair': 16, 'scooter': 13, 'skiprope': 83},
 'IL': {'blender': 105,
  'swing': 3,
  'chair': 131,
  'scooter': 3,
  'skiprope': 703},
 'TN': {'blender': 84,
  'swing': 3,
  'chair': 104,
  'scooter': 2,
  'skiprope': 560},
 'WA': {'blender': 20, 'swing': 1, 'chair': 25, 'scooter': 1, 'skiprope': 133},
 'TX': {'blender': 74, 'swing': 3, 'chair': 92, 'scooter': 2, 'skiprope': 493},
 'UT': {'blender': 69, 'swing': 3, 'chair': 87, 'scooter': 2, 'skiprope': 464}}

In [12]:
total_storage_costs = pm.storage_costs(volume_dc, pm.warehousing_storageCost)
total_storage_costs

1341612

In [13]:
total_handling_in_costs = pm.handling_in_costs(dc_product_demand_container, pm.warehousing_handlingIn_data)
total_handling_in_costs

55780

In [15]:
total_handling_out_costs = pm.handling_out_costs(as_is_dc, pm.unit_data, pm.warehousing_handlingOut_data)
total_handling_out_costs

686104.2999999999

In [20]:
class Warehousing():
    def __init__(self, dc_allocation):
        self.dc_allocation = dc_allocation

        dc_product_demand = pm.product_demand_per_dc(self.dc_allocation, pm.demand_data, pm.product_data)
        volume_dc = pm.volume_per_dc(dc_product_demand)
        storage = pm.storage_costs(volume_dc, pm.warehousing_storageCost)
        handling_in = pm.handling_in_costs(dc_product_demand_container, pm.warehousing_handlingIn_data)
        handling_out = pm.handling_out_costs(self.dc_allocation, pm.unit_data, pm.warehousing_handlingOut_data)
        warehousing = pm.warehousing_costs(storage, handling_in, handling_out)